In [53]:
import sys
import os
from pprint import pprint
import pandas as pd
import json
from tqdm import tqdm
import hashlib
import yaml
sys.path.append(os.path.abspath('../'))
from RAG.rag_utils import load_config, load_documents, split_documents
from RAG.rag_utils import get_embedding_model, make_custom_prompt, get_llm
from RAG.rag_utils import get_memory, get_vectorstore, get_multi_vectorstore
from RAG.rag_utils import get_multi_query_retriever, get_compression_retriever
from RAG.rag_utils import get_BM25_retriever, get_ensemble_retriever, get_reorder_retriever 
from RAG.rag_utils import get_conv_chain

In [54]:
def load_exam():
    """
    Reads topics from txt files and adds to a dataframe. Each topic can be in a separate file
    """
    exam = []
    #path where we store the exam topic txt files
    exam_topics_path = os.getenv("EXAM_TOPICS")

    #read in the exam topic files and return as a pandas DataFrame
    for file in list(filter(lambda f: f.endswith(".json"),os.listdir(exam_topics_path))):
        with open(os.path.join(exam_topics_path,file),'r') as file:
            exam.append(json.loads(file.read()))
    exam = pd.DataFrame(exam)
    return exam
    

In [55]:
def take_exam(exam, qa_chain):
    
    student_answers = []
    for topic_idx, topic in tqdm(exam.iterrows()):
        q1 = topic["Question 1"]
        a1 = topic["Teacher Answer 1"]
        answer1 = qa_chain({"domain":"HR and workforce transformation","question":q1,"chat_history":""})["answer"]
    
        q2 = topic["Question 2"]
        a2 = topic["Teacher Answer 2"]
        answer2 = qa_chain({"domain":"HR and workforce transformation","question":q2,"chat_history":""})["answer"]
    
        q3 = topic["Question 3"]
        a3 = topic["Teacher Answer 3"]
        answer3 = qa_chain({"domain":"HR and workforce transformation","question":q3,"chat_history":""})["answer"]
    
        q4 = topic["Question 4"]
        a4 = topic["Teacher Answer 4"]
        answer4 = qa_chain({"domain":"HR and workforce transformation","question":q4,"chat_history":""})["answer"]
        
        student_answers.append({"Question 1":q1,"Teacher Answer 1":a1,"Student Answer 1":answer1,
                                "Question 2":q2,"Teacher Answer 2":a2,"Student Answer 2":answer2,
                                "Question 3":q3,"Teacher Answer 3":a3,"Student Answer 3":answer3,
                                "Question 4":q4,"Teacher Answer 4":a4,"Student Answer 4":answer4,
                                "topic_hash":topic['topic_hash'],"topic":topic['topic']
                            })
    return pd.DataFrame(student_answers)

In [56]:
def write_exam_results(student_answers, config):
    student_id = hashlib.md5(yaml.dump(config).encode()).hexdigest()
    path = os.path.join(os.getenv("EXAM_ANSWERS"),student_id)
    student_answers.to_csv(os.path.join(path,student_answers))
    if not os.path.exists(path):
        os.makedirs(path)
    yaml_path = os.path.join(path,"config.yaml")
    with open(yaml_path,"w") as f:
        yaml.dump(config, f)
    

In [57]:
config = load_config()
config

{'random_seed': 1,
 'temperature': 0,
 'device': 'mps',
 'recreate_vector_store': False,
 'gradio_server_port': 7860,
 'chatbot_prompt': 'skills_edge_prompt_2.txt',
 'student_prompt': 'student_answer_exam_prompt.txt',
 'max_tokens': 1024,
 'chunk_size': 500,
 'chunk_overlap': 40,
 'embedding_model_name': 'thenlper/gte-large',
 'top_k': 15,
 'search_type': 'mmr',
 'score_threshold': 0.8,
 'chain_type': 'stuff',
 'multi_query': False,
 'ensemble': False,
 'bm25_retriever_k': 2,
 'compression': False,
 'multi_vector': False,
 'rephrase_question': False,
 'reorder': False}

## Load documents

In [58]:
documents = load_documents()

## Split documents

In [59]:
chunk_size = config['chunk_size']
chunk_overlap = config['chunk_overlap']
chunks = split_documents(documents, chunk_size, chunk_overlap)
print(f"""split documents with chunk size {chunk_size}, chunk_overlap {chunk_overlap}""")

split documents with chunk size 500, chunk_overlap 40


## Load embedding model

In [60]:
embedding_model_name = config["embedding_model_name"]
device = config["device"]

embeddings = get_embedding_model(embedding_model_name, device)
print(f"""Loading model {embedding_model_name}, on device {device}""")

Loading model thenlper/gte-large, on device mps


## Create a custom prompt

In [61]:
prompt_fname = config["student_prompt"]
prompt = make_custom_prompt(prompt_fname, input_variables=["domain","context","question"])

input_variables=['context', 'domain', 'question'] template='You are an expert in the domain of {domain}. To prove your expertise and add value to our project \nI would like you to do your best to answer a question from the domain. \nHere are you instructions\n1. The topic appears enclosed in triple backticks in the section Topic: after these instructions.\n2. The question appears enclosed in triple backticks in the section Teacher Question: after these instructions.\n3. Place your answer in the section Student Answer:\n\nTopic: ```{context}```\nTeacher Question: ```{question}```\nStudent:\n\n\n\n\n'


## Get memory - maybe not needed for exam?

In [62]:
memory = get_memory()

## Get LLM

In [63]:
temperature = config['temperature']
max_tokens = config['max_tokens']
random_seed = config['random_seed']
llm = get_llm(temperature, max_tokens, random_seed)
print(f"""Connect to LLM running on  {os.getenv("LLM_URL")} with temperature {temperature} and max_tokens {max_tokens}""")

Connect to LLM running on  http://localhost:1234/v1 with temperature 0 and max_tokens 1024


# Retrievers

## Base vector store retriever

In [64]:
search_type = config['search_type']
k = config['top_k']
multi_vector = config['multi_vector']

if multi_vector:
    retriever = get_multi_vectorstore(documents,
                                      embeddings,
                                      k,
                                      search_type
                                     )
else:
    retriever = get_vectorstore(chunks,
                                embeddings,
                                k,
                                search_type
                               )
print(f"""Created base retriever with search type {search_type} returning top {k} results""")

Created base retriever with search type mmr returning top 15 results


## Apply multiquery to retriever

In [65]:
multi_query = config['multi_query']
if multi_query:
    retriever = get_multi_query_retriever(llm, retriever)
print(f"""Applied multi_query to retriever: {multi_query}""")

Applied multi_query to retriever: False


## Apply compression to retriever

In [66]:
ensemble = config['ensemble']
if ensemble:
    
    bm25_retriever = get_BM25_retriever(chunks, k=2)
    retrievers=[bm25_retriever, retriever]
    
    retriever = get_ensemble_retriever(retrievers,weights=[0.5,0.5])
print(f"""Applied ensembling to retriever: {ensemble}""")

Applied ensembling to retriever: False


## Apply re-order to retriever

In [67]:
reorder = config['reorder']
if reorder:
    retriever = get_reorder_retriever(retriever)
print(f"""Applied re-order to retriever: {reorder}""")

Applied re-order to retriever: False


## Create QA Chain

In [68]:
chain_type = config["chain_type"]
rephrase_question = config["rephrase_question"]
#chain_type="refine"
memory=None

qa_chain = get_conv_chain(llm, retriever, memory, prompt, chain_type, rephrase_question)
print(f"""Created chain with chain_type: {chain_type}""")

input_variables=['context', 'domain', 'question'] template='You are an expert in the domain of {domain}. To prove your expertise and add value to our project \nI would like you to do your best to answer a question from the domain. \nHere are you instructions\n1. The topic appears enclosed in triple backticks in the section Topic: after these instructions.\n2. The question appears enclosed in triple backticks in the section Teacher Question: after these instructions.\n3. Place your answer in the section Student Answer:\n\nTopic: ```{context}```\nTeacher Question: ```{question}```\nStudent:\n\n\n\n\n'
Created chain with chain_type: stuff


In [69]:
print(f"""Running QA Chain with config \n""")
pprint(config)

Running QA Chain with config 

{'bm25_retriever_k': 2,
 'chain_type': 'stuff',
 'chatbot_prompt': 'skills_edge_prompt_2.txt',
 'chunk_overlap': 40,
 'chunk_size': 500,
 'compression': False,
 'device': 'mps',
 'embedding_model_name': 'thenlper/gte-large',
 'ensemble': False,
 'gradio_server_port': 7860,
 'max_tokens': 1024,
 'multi_query': False,
 'multi_vector': False,
 'random_seed': 1,
 'recreate_vector_store': False,
 'reorder': False,
 'rephrase_question': False,
 'score_threshold': 0.8,
 'search_type': 'mmr',
 'student_prompt': 'student_answer_exam_prompt.txt',
 'temperature': 0,
 'top_k': 15}


# Take the exam

## Load the exam questions

In [70]:
exam = load_exam()

In [72]:
student_answers = take_exam(exam,qa_chain)

0it [00:00, ?it/s]/opt/homebrew/Caskroom/miniforge/base/envs/sem-py311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


A well-rounded Skills Library is essential for defining roles effectively and ensuring that organizations have the necessary workforce capabilities to meet their business needs. Including a variety of traditional categories such as competencies, qualifications, certifications, technical and behavioral skills, and region-specific expectations in a Skills Library offers several benefits:

1. Comprehensive understanding of roles: By incorporating various categories, organizations can gain a more complete understanding of the skills required for each role. This holistic approach allows them to account for both hard (technical) and soft (behavioral) skills, as well as any region-specific knowledge or qualifications that may be necessary.
2. Flexibility in workforce planning: A Skills Library with a diverse range of categories enables organizations to adapt to changing business needs and market conditions. For instance, they can identify the skills needed for new roles or projects and ensure

1it [01:21, 81.17s/it]

The 'Skills over Degrees' initiative refers to a talent strategy where organizations prioritize hiring individuals based on their skills rather than their educational background. IBM is one of the major companies that has been well-documented for implementing this approach. This shift towards skills-based hiring is driven by the increasing demand for in-demand skills and the need for companies to stay competitive in the market. Additionally, technology can help light the way for employees to assess their skills portfolio against market needs and choose the right upskilling opportunities. However, many companies face challenges in tracking the availability of internal and external skills, making it essential to invest in tools and technologies that enable easy updates and collaboration on skills taxonomies.Based on the information provided in the text, 45% of companies report that their need for in-demand skills is higher than for other skills. This makes talent search and retention eve

2it [02:49, 85.44s/it]

...

12it [14:35, 70.27s/it]

The decrease in interest in tracking skills within Production, Supply Chain, and Creative & Design job families can be attributed to the increasing automation and robotization of roles in these areas. As more tasks become automated, the need for human skills in traditional manufacturing and supply chain roles may become less priority. However, it's important to note that this trend does not necessarily mean that skills are no longer valuable in these job families. Instead, the types of skills required may shift towards areas such as problem-solving, adaptability, and technical expertise related to automation and robotics. Additionally, creative roles may require a greater focus on soft skills like creativity, communication, and collaboration as technology continues to advance. Ultimately, organizations need to keep up with the changing demands of these job families and invest in reskilling and upskilling their workforce to remain competitive.The rise of automation and robotics is leadi

13it [15:37, 67.96s/it]

In today's business environment, organizations are striving to future-proof themselves by preparing their workforces for the upheavals that may come. This involves aligning skills with future strategy and identifying critical work that will contribute to the organization's long-term success. Future-proofing an organization can be achieved through various use cases such as reskilling/upskilling the workforce, unlocking trapped capacity, matching work demand with availability, responding to employee needs, and empowering employees to choose opportunities in line with their aspirations.

One of the key aspects of future-proofing is predicting the skills of the future and investing in them before they become essential. This requires organizations to have a clear understanding of how work will change and the impact on skills demand. By doing so, they can reskill or upskill their people to be ready for the future and locate the right external or internal talent to fit future needs.

However,

14it [16:49, 72.14s/it]


student_answers = take_exam(exam, qa_chain)

In [73]:
write_exam_results(student_answers, config)

In [74]:
type(student_answers)

pandas.core.frame.DataFrame